# Xarray-simlab: run models and visualize outputs

We'll see here how to:

- setup and run one simulation
- save and reuse simulation setups
- set time-varying input values (external forcing)
- save model variable snapshots at different time steps / time frequencies
- save model snapshots to different stores (e.g., in-memory vs. on-disk) and save model outputs to a file
- run simulation batches in parallel
- leverage some xarray features for simulation pre/post-processing
- visualize and compare (interactively) simulation outputs using xarray plotting features (built on top of matplotlib), hvplot (holoviews/bokeh) and ipyfastscape (ipywidgets)

In [ ]:
import numpy as np
import xsimlab as xs
import xarray as xr

# visualization
import matplotlib as plt
import hvplot.xarray
from ipyfastscape import TopoViz3d

# ipython magic command
%load_ext xsimlab.ipython

## Setup a new simulation from scratch

Let's import a fastscape model

In [ ]:
from fastscape.models import basic_model

When used in jupyter notebooks or consoles, xarray-simlab provides a convenient "magic" command `%create_setup` that can be used to generate some template code to help create a new setup for a given model. This template may be pre-filled with variables default values and/or some documentation (e.g., input description) as line comments.

This template has to be filled with missing input values, "clocks" (i.e., coordinates for the simulation time steps and variable snapshot saving) and output variables. More details can be found in xarray-simlab's documentation.

In [ ]:
# %create_setup basic_model
import xsimlab as xs

ds_in = xs.create_setup(
    model=basic_model,
    clocks={
        'time': np.linspace(0., 1e6, 101),
        'out': np.linspace(0., 1e6, 51)
    },
    master_clock='time',
    input_vars={
        'grid__shape': [101, 201],
        'grid__length': [1e4, 2e4],
        'boundary__status': ['looped', 'looped', 'fixed_value', 'fixed_value'],
        'uplift__rate': 1e-3,
        'spl__k_coef': 1e-4,
        'diffusion__diffusivity': 1e-1,
    },
    output_vars={
        'topography__elevation': 'out',
        'drainage__area': 'out',
        'flow__basin': 'out',
        'spl__chi': 'out',
    }
)


The `xs.create_setup` function returns a new `xarray.Dataset` object. The latter contains the whole simulation setup (data + metadata) saved into a tidy data structure (see xarray's documentation for an overview of what can be done with xarray datasets)

In [ ]:
ds_in

### Save / load a simulation setup to / from a file

The dataset created above can be saved to a netCDF file:

In [ ]:
ds_in.to_netcdf('sim_setup.nc')

This is useful when we want to reuse a simulation setup in a different context. NetCDF is a common, portable format used in geosciences.

In [ ]:
ds_in_reloaded = xr.load_dataset('sim_setup.nc')

ds_in_reloaded

## Run a simulation

Xarray-simlab provides an `.xsimlab` dataset "accessor" that extends its functionality with simulation specfic operations. Just use `.xsimlab.run()` applied on the input dataset created above. Note: it is important to provide the same xarray-simlab same model that has been used to create the input dataset.

In [ ]:
ds_out = ds_in.xsimlab.run(model=basic_model)

The model object can also be used as a "context" (i.e., using the `with` statement). In the code block of this context, it is not needed to set the `model` argument for the functions available in the `.xsimlab` dataset accessor.

In [ ]:
with basic_model:
    ds_out = ds_in.xsimlab.run()

Xarray-simlab provides helpful tools for monitoring model runs, e.g., a progress bar (that can be used as a context too):

In [ ]:
with basic_model, xs.monitoring.ProgressBar():
    ds_out = ds_in.xsimlab.run()

`.xsimlab.run()` returns a new xarray Dataset object, with both model input and outputs:

In [ ]:
ds_out

### Save simulation outputs to a file

Like the input dataset, we can save the output dataset to a netCDF file.

In [ ]:
# the netcdf file format has some limitations on data format encodings
# (needs an encoding fix in fastscape)
ds_out['border'] = ds_out.coords['border'].astype('S6')

ds_out.to_netcdf('sim_output.nc')

### Save simulation outputs to a (zarr) store as the simulation proceeds

By default, all model output snapshots are saved into memory before returning it as a xarray Dataset. This might be problematic when running large models and/or saving snapshots very frequently (e.g., at each time step). Fortunately, Xarray-simlab may use alternative storage solutions for storing the outputs. This functionality is built on top of the [zarr](https://zarr.readthedocs.io) library, which provides many storage options like filesystem directories, databases, distributed cloud storage systems, etc. It also supports many compression options.

In the example below, we store model output snapshots into a directory named `sim_output.zarr`:

In [ ]:
with basic_model, xs.monitoring.ProgressBar():
    ds_out = ds_in.xsimlab.run(store='sim_output.zarr')

Let's inspect the model outputs saved in the zarr store:

In [ ]:
import zarr

zdataset = zarr.open('sim_output.zarr')

zdataset.info

The compression ratio is much better for the `flow__basin` output variable than for the `topography__elevation` variable. This is likely explained by the data value patterns on the grid.

In [ ]:
zdataset.flow__basin.info

In [ ]:
zdataset.topography__elevation.info

At the end of the simulation, the zarr store is loaded back as a xarray dataset. Note that it is loaded "lazily" (no data is actually loaded in-memory). This allows dealing with large amount of data represented as "logical" data cubes.

In [ ]:
ds_out

The data is also chunked (dask arrays), which enables efficient operations (post-processing or visualization) computed in parallel. This is not always desirable, though. You can load the whole dataset in-memory, if it's not too large:

In [ ]:
ds_out.nbytes / 1e6

In [ ]:
# Ok, ~30 Mb is not very much

ds_out.load()

### Simulation outputs post-processing and visualization 

Xarray allows to easily and efficiently perform many kinds of operations on datasets (e.g., selection, filtering, arithmetics, aggregations, visualization, etc.)

For example, let's select the last snapshot saved:

In [ ]:
ds_out.isel(out=-1)

Or let's select the snashot based on a given simulation (absolute) time:

In [ ]:
# nearest means select the nearest value
# (otherwise, raises an error if no snapshot has been saved for the given time)
ds_out.sel(out=5.3e5, method='nearest')

Let's plot the elevation data for the last snapshot:

In [ ]:
ds_out.isel(out=-1).topography__elevation.plot(aspect=2, size=4);

Let's plot several snapshots:

In [ ]:
(
    ds_out
    .sel(out=[0., 2.5e5, 5e5, 1e6], method='nearest')
    .topography__elevation
    .plot(col='out', col_wrap=2, aspect=2, size=3)
)

We can also extract cross-sections:

In [ ]:
(
    ds_out
    .sel(x=[0., 1e4, 2e4])
    .sel(out=[0., 2.5e5, 5e5, 1e6], method='nearest')
    .topography__elevation
    .plot(col='out', hue='x', col_wrap=2, aspect=1, size=3)
)

For interactive plots, we can use the hvplot library:

In [ ]:
ds_out.topography__elevation.hvplot(x='x', y='y', groupby='out', data_aspect=1)

In [ ]:
(
    ds_out
    .sel(x=[0., 1e4, 2e4])
    .sel(out=[0., 2.5e5, 5e5, 1e6], method='nearest')
    .topography__elevation
    .hvplot(x='y', groupby='out', by='x', aspect=2, ylim=[0, 400])
)

For 3D visualization, we can use the ipyfastscape library that provides an interface similar to Paraview.

In [ ]:
app = TopoViz3d(ds_out, time_dim='out')

app.show()

## Reuse previous simulation settings for running new simulations

- update input vars (just a new value for one input)
- update the model used (use MultipleFlowRouter)

## Set time-varying input values

- example with k_coef

## Exercise: effect of erosion magnitude on domain-integrated sediment fluxes

- Re-run a new simulation with time varying erosion coefficients. Save snapshots for the erosion rate (all processes)
- Compute the erosion rate integrated over the whole domain using xarray (you can get the grid total area as a model output so you don't need to recalculate it)
- Plot time series for both the erosion coefficients and the sediment fluxes, using xarray's plotting functions

## Run simulation batches

- Multiple values for k_coef



### Compare model runs

## Exercise: leverage xarray's n-dimensional datasets

- set and run a batch of simulations for different (array) values of SPL's K coefficient
- for each simulation, K must be variable in both space and time
   - the input variable for K should have 4 dimensions: 'batch', 'x', 'y', 'time'
- plot the values of K with xarray (facetting)
- compare the model runs with hvplot or ipyfastscaoe